# Ratios - Growth

In [1]:
# Import necessary packages
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

In [2]:
# Read nifty 50 index data
df = pd.read_csv('./data/annual_reports/nifty_50.csv')

In [3]:
df.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
1,Adani Enterprises Ltd.,METALS,ADANIENT,EQ,INE423A01024
2,Adani Green Energy Ltd.,POWER,ADANIGREEN,EQ,INE364U01010
3,Adani Transmission Ltd.,POWER,ADANITRANS,BE,INE931S01010
4,Ambuja Cements Ltd.,CEMENT & CEMENT PRODUCTS,AMBUJACEM,EQ,INE079A01024


In [4]:
# Number of companies by industry sectors in nifty 50.
df['Industry'].value_counts()

FINANCIAL SERVICES          12
CONSUMER GOODS               8
PHARMA                       6
METALS                       5
CONSUMER SERVICES            3
OIL & GAS                    3
CEMENT & CEMENT PRODUCTS     2
POWER                        2
SERVICES                     1
CHEMICALS                    1
FERTILISERS & PESTICIDES     1
IT                           1
AUTOMOBILE                   1
TELECOM                      1
CONSTRUCTION                 1
HEALTHCARE SERVICES          1
INDUSTRIAL MANUFACTURING     1
Name: Industry, dtype: int64

Nifty 50 index is a heavy investor in finance, followed by consumer goods and pharmaceuticals. These 3 sectors contribute a little more than half of the campnies in the index.

## Year-to-year Growth Rate

Year-to-year growth rate is the rate of change in the specific metric of interest this year with respect to previous year. Lets us work on an example. Considered total revenue for understanding about growth rate. Note that other values in income statement can also be considered for measuring growth. Mathematically, year-to-year growth rate is represented as 
$$
YoY = \bigg(\frac{Statistic_{n+1} - Statistic_{n}}{Statistic_{n}}\bigg)
$$

### Example: Total Revenue

In [5]:
yoy_df = pd.DataFrame(columns=["Industry", "2021", "2020", "2019"])
years = [2021, 2020, 2019]

In [6]:
# Calculating year-to-year growth for companies in nifty 50.
for index, row in df.iterrows():
    ticker = row["Symbol"]
    income_stmt_path = f"./data/annual_reports/{ticker}/income_stmt.csv"
    is_df = pd.read_csv(income_stmt_path, index_col=[0])
    is_df_cols = [col.split("-")[0] for col in is_df.columns]
    is_df.columns = is_df_cols
    revenue = is_df.loc["Total Revenue"]
    for _, year in enumerate(years):
        if f"{year}" in revenue.index:
            yoy_df.at[str(ticker), str(year)] = (revenue[f"{year}"] - revenue[f"{year-1}"])*100/revenue[f"{year-1}"]
    yoy_df.at[str(ticker), "Industry"] = row["Industry"]
    yoy_df.at[str(ticker), "Company Name"] = row["Company Name"]

In [7]:
yoy_df.head()

,Industry,2021,2020,2019,Company Name
ACC,CEMENT & CEMENT PRODUCTS,NaN,-11.953147,5.782678,ACC Ltd.
ADANIENT,METALS,-8.905995,7.488857,12.400484,Adani Enterprises Ltd.
ADANIGREEN,POWER,22.705882,22.770896,40.313752,Adani Green Energy Ltd.
ADANITRANS,POWER,-13.048662,56.266349,85.206922,Adani Transmission Ltd.
AMBUJACEM,CEMENT & CEMENT PRODUCTS,NaN,-9.546277,4.080536,Ambuja Cements Ltd.


In [8]:
# Determine top 5 companies with highest average year-to-year growth in last 3 years.
temp_yoy_df = yoy_df.copy()
temp_yoy_df["2021-2020"] = temp_yoy_df["2021"] - temp_yoy_df["2020"]
temp_yoy_df["2020-2019"] = temp_yoy_df["2020"] - temp_yoy_df["2019"]
temp_yoy_df["yoy_mean"] = (temp_yoy_df["2021-2020"] + temp_yoy_df["2020-2019"])/2
temp_yoy_df.dropna(axis=0, inplace=True)
temp_yoy_df.sort_values("yoy_mean", ascending=False, inplace=True)

In [15]:
temp_yoy_df_display = temp_yoy_df[["Company Name", "Industry", "yoy_mean"]]
temp_yoy_df_display.columns = ["Company Name", "Industry", "Average YoY"]
temp_yoy_df_display.head()

,Company Name,Industry,Average YoY
ICICIPRULI,ICICI Prudential Life Insurance Company Ltd.,FINANCIAL SERVICES,135.8033
INDUSTOWER,Indus Towers Ltd.,TELECOM,51.924432
SIEMENS,Siemens Ltd.,INDUSTRIAL MANUFACTURING,17.435053
NMDC,NMDC Ltd.,METALS,13.373396
PNB,Punjab National Bank,FINANCIAL SERVICES,12.006564


The above are the top 5 companies in Nifty 50 index with highest average year-to-year growth in the last 3 years. ICICI prudential being the top contender with 135% average year-to-year growth in last 3 years.

In [10]:
sector_yoy = temp_yoy_df.groupby(['Industry']).mean()
sector_yoy

/var/folders/zl/bqtzmz3s0xl5_ddgbjsqxsww0000gn/T/ipykernel_38414/1483552614.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  sector_yoy = temp_yoy_df.groupby(['Industry']).mean()


,2021,2020,2019,2021-2020,2020-2019,yoy_mean
Industry,,,,,,
AUTOMOBILE,-0.287075,-19.621105,3.487628,19.334030,-23.108733,-1.887351
CHEMICALS,-0.024128,3.058932,16.444267,-3.083060,-13.385335,-8.234198
CONSTRUCTION,-10.993527,-27.292508,24.740576,16.298981,-52.033083,-17.867051
CONSUMER GOODS,7.791886,-0.042512,13.416183,7.834399,-13.458695,-2.812148
CONSUMER SERVICES,-11.071530,16.174625,22.529155,-27.246155,-6.354530,-16.800342
FERTILISERS & PESTICIDES,35.957226,18.501179,24.759563,17.456046,-6.258383,5.598832
FINANCIAL SERVICES,26.025135,35.959907,21.045491,-9.934772,14.914417,2.489822
HEALTHCARE SERVICES,-6.106626,16.940993,16.668130,-23.047619,0.272863,-11.387378
INDUSTRIAL MANUFACTURING,37.125622,-23.979089,2.255516,61.104711,-26.234605,17.435053


The above table summarizes the year-to-year growth for each sector of the companies present in nifty 50.

In [11]:
# Sort the result
sector_yoy.sort_values("yoy_mean", ascending=False, inplace=True)
sector_yoy.head()

,2021,2020,2019,2021-2020,2020-2019,yoy_mean
Industry,,,,,,
TELECOM,106.944980,-1.218833,3.096116,108.163813,-4.314949,51.924432
INDUSTRIAL MANUFACTURING,37.125622,-23.979089,2.255516,61.104711,-26.234605,17.435053
FERTILISERS & PESTICIDES,35.957226,18.501179,24.759563,17.456046,-6.258383,5.598832
FINANCIAL SERVICES,26.025135,35.959907,21.045491,-9.934772,14.914417,2.489822
METALS,13.263016,-6.967148,15.431101,20.230164,-22.398249,-1.084042


Telecom sector experienced the largest average year-to-year growth in the nifty 50 index. It is followed by industrial manufacturing and fertilisers & pesticides. Note that only 4 sectors reported postive average year-to-year growth, while rest all the sectors reported negative growth.

In [13]:
sector_yoy.tail()

,2021,2020,2019,2021-2020,2020-2019,yoy_mean
Industry,,,,,,
CONSUMER SERVICES,-11.071530,16.174625,22.529155,-27.246155,-6.354530,-16.800342
CONSTRUCTION,-10.993527,-27.292508,24.740576,16.298981,-52.033083,-17.867051
OIL & GAS,-19.349204,1.789523,30.781976,-21.138727,-28.992453,-25.065590
POWER,4.828610,39.518622,62.760337,-34.690012,-23.241715,-28.965863
SERVICES,-59.054059,25.473864,23.786594,-84.527923,1.687270,-41.420327


Services Industry experiences the greatest set back due to corona for nifty 50. It is followed by power and oil & gas industry.

In [12]:
# Plot the sector-wise year-to-year growth
## Note - This section will not be available in github preview as it is a interactive chart. 
## Please download the notebook and run the notebook on the local machine to play with the interactive chart.
sector1 = widgets.Dropdown(
    options=list(sector_yoy.index),
    value="TELECOM",
    description="Sector"
)
years = ["2021", "2020", "2019"]
trace1 = go.Bar(x=years, y=sector_yoy.loc["TELECOM"].values[:3])
g1 = go.FigureWidget(data=[trace1],
                    layout=go.Layout(
                        title = dict(text="Sector-Wise Year-to-year Growth Rate")
                    ))

def response1(change):
    sector_data = sector_yoy.loc[sector1.value].values[:3]
    with g1.batch_update():
        g1.data[0].y = sector_data

sector1.observe(response1, names="value")

widgets.VBox([sector1, g1])

Evalutating the top sector in nifty 50 which is financial services.

In [17]:
# Analysing YoY in financial sector companies
fin_yoy_df = yoy_df[yoy_df["Industry"] == "FINANCIAL SERVICES"]
print(f"Number of financial services companies in Nifty 50: {len(fin_yoy_df)}")
fin_yoy_df.head()

Number of financial services companies in Nifty 50: 12


,Industry,2021,2020,2019,Company Name
BAJAJHLDNG,FINANCIAL SERVICES,5.160416,0.950548,1.640078,Bajaj Holdings & Investment Ltd.
BANDHANBNK,FINANCIAL SERVICES,-11.020887,34.346156,43.371996,Bandhan Bank Ltd.
BANKBARODA,FINANCIAL SERVICES,10.118021,56.110017,55.012545,Bank of Baroda
CHOLAFIN,FINANCIAL SERVICES,21.409619,17.654542,20.03826,Cholamandalam Investment and Finance Company Ltd.
HDFCAMC,FINANCIAL SERVICES,2.771049,2.182425,12.134064,HDFC Asset Management Company Ltd.


In [25]:
fin_ind_yoy = pd.concat([fin_yoy_df[["Industry", "2021", "2020", "2019", "Company Name"]], temp_yoy_df[["yoy_mean"]]], axis=1, join="inner")
fin_ind_yoy.sort_values("yoy_mean", ascending=False, inplace=True)

In [26]:
fin_ind_yoy

,Industry,2021,2020,2019,Company Name,yoy_mean
ICICIPRULI,FINANCIAL SERVICES,279.539784,-46.857573,7.933184,ICICI Prudential Life Insurance Company Ltd.,135.8033
PNB,FINANCIAL SERVICES,81.145185,652.553038,57.132058,Punjab National Bank,12.006564
MUTHOOTFIN,FINANCIAL SERVICES,12.723261,26.580363,8.743617,Muthoot Finance Ltd.,1.989822
BAJAJHLDNG,FINANCIAL SERVICES,5.160416,0.950548,1.640078,Bajaj Holdings & Investment Ltd.,1.760169
CHOLAFIN,FINANCIAL SERVICES,21.409619,17.654542,20.03826,Cholamandalam Investment and Finance Company Ltd.,0.685679
ICICIGI,FINANCIAL SERVICES,11.748244,11.402281,20.932473,ICICI Lombard General Insurance Company Ltd.,-4.592115
HDFCAMC,FINANCIAL SERVICES,2.771049,2.182425,12.134064,HDFC Asset Management Company Ltd.,-4.681507
PEL,FINANCIAL SERVICES,-3.161593,9.913223,12.233138,Piramal Enterprises Ltd.,-7.697365
SBICARD,FINANCIAL SERVICES,7.031038,23.652119,37.486911,SBI Cards and Payment Services Ltd.,-15.227937
BANKBARODA,FINANCIAL SERVICES,10.118021,56.110017,55.012545,Bank of Baroda,-22.447262


## Compound Average Growth Rate (CAGR)

The mathematical representation of Compound Average Growth Rate (CAGR) is as follows: 
$$
CAGR = \bigg(\frac{Statistic_{n+1}}{Statistic_{n}}\bigg)^{1/t} - 1
$$

In [27]:
# Create a empty dataframe for storing CAGR values
cagr_df = pd.DataFrame(columns=["2021", "2020", "2019"])
years = [2021, 2020, 2019]

In [37]:
# Calculating CAGR for companies in nifty 50.
for index, row in df.iterrows():
    ticker = row["Symbol"]
    income_stmt_path = f"./data/annual_reports/{ticker}/income_stmt.csv"
    is_df = pd.read_csv(income_stmt_path, index_col=[0])
    is_df_cols = [col.split("-")[0] for col in is_df.columns]
    is_df.columns = is_df_cols
    revenue = is_df.loc["Total Revenue"]
    for _, year in enumerate(years):
        if f"{year}" in revenue.index:
            cagr_df.at[str(ticker), str(year)] = (((revenue[f"{year}"]/revenue["2018"])**(1/(year-2018)))-1)*100
    cagr_df.at[str(ticker), "Industry"] = row["Industry"]
    cagr_df.at[str(ticker), "Company Name"] = row["Company Name"]

/var/folders/zl/bqtzmz3s0xl5_ddgbjsqxsww0000gn/T/ipykernel_38414/2159808290.py:11: RuntimeWarning:

invalid value encountered in double_scalars



In [38]:
cagr_df.head()

,2021,2020,2019,Industry,Company Name
PNB,177.721542,243.875279,57.132058,FINANCIAL SERVICES,Punjab National Bank
BANKBARODA,38.638975,55.560313,55.012545,FINANCIAL SERVICES,Bank of Baroda
ADANITRANS,36.019023,70.122337,85.206922,POWER,Adani Transmission Ltd.
ICICIPRULI,29.603847,-24.264728,7.933184,FINANCIAL SERVICES,ICICI Prudential Life Insurance Company Ltd.
GLAND,28.8188,27.495725,26.189966,PHARMA,Gland Pharma Ltd.


In [45]:
cagr_df.sort_values("2021", ascending=False, inplace=True)
cagr_df

,2021,2020,2019,Industry,Company Name
PNB,177.721542,243.875279,57.132058,FINANCIAL SERVICES,Punjab National Bank
BANKBARODA,38.638975,55.560313,55.012545,FINANCIAL SERVICES,Bank of Baroda
ADANITRANS,36.019023,70.122337,85.206922,POWER,Adani Transmission Ltd.
ICICIPRULI,29.603847,-24.264728,7.933184,FINANCIAL SERVICES,ICICI Prudential Life Insurance Company Ltd.
GLAND,28.8188,27.495725,26.189966,PHARMA,Gland Pharma Ltd.
ADANIGREEN,28.337532,31.249553,40.313752,POWER,Adani Green Energy Ltd.
INDUSTOWER,28.210556,0.915581,3.096116,TELECOM,Indus Towers Ltd.
PIIND,26.202009,21.590112,24.759563,FERTILISERS & PESTICIDES,PI Industries Ltd.
SBICARD,22.083663,30.386149,37.486911,FINANCIAL SERVICES,SBI Cards and Payment Services Ltd.
BIOCON,19.890794,23.611756,33.734297,PHARMA,Biocon Ltd.


3 of the top highest growth companies belongs to the financial services sector. Punjab National Bank leads in the index with a massive 2042% CAGR.

In [40]:
# Calculate CAGR sector-wise and sort the values with decreasing order for the year 2021
sector_cagr = cagr_df.groupby(['Industry']).mean()
sector_cagr.sort_values("2021", ascending=False, inplace=True)
sector_cagr

/var/folders/zl/bqtzmz3s0xl5_ddgbjsqxsww0000gn/T/ipykernel_38414/2080455697.py:1: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.



,2021,2020,2019
Industry,,,
POWER,32.178277,50.685945,62.760337
TELECOM,28.210556,0.915581,3.096116
FERTILISERS & PESTICIDES,26.202009,21.590112,24.759563
FINANCIAL SERVICES,24.881961,37.816435,21.045491
IT,19.183947,22.020222,29.279409
PHARMA,13.439760,15.535077,18.542883
HEALTHCARE SERVICES,8.605375,16.804482,16.668130
CONSUMER SERVICES,8.170546,19.302498,22.529155
CONSUMER GOODS,6.799081,6.413495,13.416183


Like we saw in the yoy section, power sector tops the chart with 32% growth rate. It is followed by telecom, fertilisers & pesticides and financial services. Again, sectors such as services, construction, automobile, industrial manufacturing are hit significantly by the pandemic.

In [44]:
# Plot the sector-wise year-to-year growth
## Note - This section will not be available in github preview as it is a interactive chart. 
## Please download the notebook and run the notebook on the local machine to play with the interactive chart.
sector2 = widgets.Dropdown(
    options=list(sector_cagr.index),
    value="POWER",
    description="Sector"
)
years = ["2021", "2020", "2019"]
trace2 = go.Bar(x=years, y=sector_cagr.loc["POWER"].values)
g2 = go.FigureWidget(data=[trace2],
                    layout=go.Layout(
                        title = dict(text="Sector-Wise CAGR")
                    ))

def response2(change):
    sector_data = sector_cagr.loc[sector2.value].values
    with g2.batch_update():
        g2.data[0].y = sector_data

sector2.observe(response2, names="value")

widgets.VBox([sector2, g2])

In [46]:
# Analysing CAGR in financial sector companies
cagr_df[cagr_df["Industry"] == "FINANCIAL SERVICES"]

,2021,2020,2019,Industry,Company Name
PNB,177.721542,243.875279,57.132058,FINANCIAL SERVICES,Punjab National Bank
BANKBARODA,38.638975,55.560313,55.012545,FINANCIAL SERVICES,Bank of Baroda
ICICIPRULI,29.603847,-24.264728,7.933184,FINANCIAL SERVICES,ICICI Prudential Life Insurance Company Ltd.
SBICARD,22.083663,30.386149,37.486911,FINANCIAL SERVICES,SBI Cards and Payment Services Ltd.
CHOLAFIN,19.690724,18.840424,20.03826,FINANCIAL SERVICES,Cholamandalam Investment and Finance Company Ltd.
BANDHANBNK,19.672,38.785722,43.371996,FINANCIAL SERVICES,Bandhan Bank Ltd.
MUTHOOTFIN,15.769605,17.323512,8.743617,FINANCIAL SERVICES,Muthoot Finance Ltd.
ICICIGI,14.610896,16.069606,20.932473,FINANCIAL SERVICES,ICICI Lombard General Insurance Company Ltd.
PEL,6.105907,11.067123,12.233138,FINANCIAL SERVICES,Piramal Enterprises Ltd.
HDFCAMC,5.599417,7.042658,12.134064,FINANCIAL SERVICES,HDFC Asset Management Company Ltd.


As can be seen, averaging year-to-year growth gives a different sense of movement than compound average growth rate.